<a href="https://colab.research.google.com/github/muxeres/Analise-Fraude-Bancario/blob/master/Copia_de_an%C3%A1lise_de_roubo_e_furto_de_ve%C3%ADculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
luanamartinsilva_bos_sp_roubofurto_de_veculo_2019_path = kagglehub.dataset_download('luanamartinsilva/bos-sp-roubofurto-de-veculo-2019')
luanamartinsilva_municipios_sp_ibge_path = kagglehub.dataset_download('luanamartinsilva/municipios-sp-ibge')
luanamartinsilva_frota_de_veculos_2019_sp_path = kagglehub.dataset_download('luanamartinsilva/frota-de-veculos-2019-sp')
luanamartinsilva_sp_municpios_geopandas_path = kagglehub.dataset_download('luanamartinsilva/sp-municpios-geopandas')
luanamartinsilva_feriados_e_dias_da_semana_brasil_path = kagglehub.dataset_download('luanamartinsilva/feriados-e-dias-da-semana-brasil')

print('Data source import complete.')


Fonte de dados reais para gerar os dados Fakes

https://www.denatran.gov.br/component/content/article/115-portal-denatran/8551-estatisticas-denatran.html

Fonte dos metadados das bases de dados do RENAVAM

https://ck.govdata.gov.br/pt_BR/dataset/renavam

**Conceitos**:

*   Roubo de veículo: subtração, com violência ou grave ameaça, de veículo automotor

*   Furto de veículo: subtração, sem violência ou grave ameaça, de veículo automotor


In [ ]:
!pip install pandas --upgrade
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from shapely.geometry import Point
import geopandas as gpd
from sklearn import metrics
#import datetime
%matplotlib inline

In [ ]:
from platform import python_version
print(python_version())
!pip show pandas
!pip show numpy
!pip show seaborn

In [ ]:
#função para extrair dataframe de arquivo xls da url
import requests
def get_dataframe_from_url(url, sheet_name = 0, skiprows = 0):
  r = requests.get(url, verify=False)
  with io.BytesIO(r.content) as fh:
    return pd.read_excel(fh,sheet_name=sheet_name, skiprows=skiprows)


In [ ]:
#função para extrair dataframe de arquivo xls local
def get_dataframe_from_file(file, sheet_name = 0, skiprows = 0):
    return pd.read_excel(file,sheet_name=sheet_name, skiprows=skiprows)

In [ ]:
#pré-processamento do dataframe
def get_prepared_dataframe(dataset, weight_columns, value_colums) :
  columns = weight_columns + value_colums
  #mantem apenas as colunas necessárias
  dataset = dataset[columns]
  #eliminando dados sem informação
  dataset.dropna(inplace=True)
  #dataset = dataset[dataset[weight_columns] != 'Não se Aplica']
  #dataset = dataset[dataset[weight_columns] != 'Não Identificado']
  #dataset = dataset[dataset[weight_columns] != 'Sem Informação']
  dataset.set_index(weight_columns, inplace=True)

  return dataset

In [ ]:
#função para gerar array de pesos
def get_weights(dataset):
  total = dataset.sum()
  weights = [ round(item / total, 4) for item in dataset.values]
  weights = np.asarray(weights)
  weights = weights.flatten()
  # adicionando a diferença para a soma das probabilidade dar 1
  weights[0] =  weights[0] + (1 - weights.sum())
  return weights

In [ ]:
#função para gerar array do domínio de valores
def get_values(dataset):
  values = dataset.index
  return values.to_numpy()

In [ ]:
#função para gerar os dados fictícios
def get_fake_values(dataset, fake_size):
  weights = get_weights(dataset)
  values = get_values(dataset)
#Gerando um dataset para vizualizar melhor as probabilidades
  probability_df = pd.DataFrame(weights, index = values)
  probability_df.sort_values(by=0, ascending=False, inplace=True)
  probability_df.head()
  np.random.seed(10)
  fake_values = np.random.choice(values, p=weights, size=fake_size)
  return probability_df, fake_values

# Carregando informações da frota de São Paulo em Dezembro 2019 por município

## Ano Modelo e ano fabricação

In [ ]:
frota_ano_file = "https://infraestrutura.gov.br/images/Estatistica/RENAVAM/2019/Dezembro/F_Frota_por_UF_Municipio_ANO_FAB_MOD_Dezembro_2019.xlsx"
frota_ano = get_dataframe_from_file(frota_ano_file)
frota_ano = frota_ano[frota_ano.UF == 'SAO PAULO']
frota_ano = get_prepared_dataframe(frota_ano, ['Ano Modelo', 'Ano Fabricação CRV'],['Município','Qtd. Veículos'])
frota_ano.head()

## Tipo veículo e espécie

In [ ]:
#frota_tipo_especie_file = "https://infraestrutura.gov.br/images/Estatistica/RENAVAM/2019/Dezembro/G_Frota_por_UF_Municipio_TIPO_ESPECIE_EIXOS_Dezembro-2019.xlsx"
frota_tipo_especie_file = "../input/frota-de-veculos-2019-sp/G_Frota_por_UF_Municipio_TIPO_ESPECIE_EIXOS_Dezembro-2019.xlsx"
frota_tipo_especie = get_dataframe_from_file(frota_tipo_especie_file)
frota_tipo_especie = frota_tipo_especie[frota_tipo_especie.UF == 'SAO PAULO']
frota_tipo_especie = get_prepared_dataframe(frota_tipo_especie, ['Tipo Veículo', 'Espécie Veículo'], ['Município','Qtd. Veículos'])
#frota_tipo_especie = frota_tipo_especie.groupby(level=[0]).sum()
frota_tipo_especie.head()

## Cores dos veículos

In [ ]:
#frota_cor_file = "https://infraestrutura.gov.br/images/Estatistica/RENAVAM/2019/Dezembro/C_Frota_por_UF_Municipio_COR_Dezembro_2019.xlsx"
frota_cor_file = "../input/frota-de-veculos-2019-sp/C_Frota_por_UF_Municipio_COR_Dezembro_2019.xlsx"
frota_cor = get_dataframe_from_file(frota_cor_file)
frota_cor = frota_cor[frota_cor.UF == 'SAO PAULO']
frota_cor = get_prepared_dataframe(frota_cor, ['Cor Veículo'],['Município','Qtd. Veículos'])
#frota_cor = frota_cor.groupby(level=[0]).sum()
frota_cor.head()

In [ ]:
frota_cor.sort_values(by="Qtd. Veículos",  ascending=False)

## Marca/modelo dos veículos

In [ ]:
frota_marca_modelo_file = "../input/frota-de-veculos-2019-sp/I_Frota_por_UF_Municipio_Marca_e_Modelo_Ano_Dezembro_2019.csv"
frota_marca_modelo = pd.read_csv(frota_marca_modelo_file, sep=';')
frota_marca_modelo = get_prepared_dataframe(frota_marca_modelo, ['Marca Modelo'], ['Município','Qtd. Veículos'])
#frota_marca_modelo = frota_marca_modelo.groupby(level=[0]).sum()
frota_marca_modelo

# Analisando estatísticas de roubo/furto de veículos em SP de Agosto a Dezembro de 2019

In [ ]:
roubo_file = '../input/bos-sp-roubofurto-de-veculo-2019/Roubo Veic Ago a Dez 2019.xlsx'
roubo_dataframe = pd.concat(pd.read_excel(roubo_file, sheet_name=None), ignore_index=True)
print(f'O dataset de roubo de veículos possui {roubo_dataframe.shape[0]} linhas e {roubo_dataframe.shape[1]} colunas.')

In [ ]:
furto_file = '../input/bos-sp-roubofurto-de-veculo-2019/Furto Veic Ago a Dez19 (1).xlsx'
furto_dataframe = pd.concat(pd.read_excel(furto_file, sheet_name=None), ignore_index=True)
print(f'O dataset de furto de veículos possui {furto_dataframe.shape[0]} linhas e {furto_dataframe.shape[1]} colunas.')

In [ ]:
frames = [roubo_dataframe, furto_dataframe]
roubo_furto_dataframe = pd.concat(frames)
print(f'O dataset de furto e roubo de veículos possui {roubo_furto_dataframe.shape[0]} linhas e {roubo_furto_dataframe.shape[1]} colunas.')

## Analisando campo 'Rúbricas'

In [ ]:
#rubricas
roubo_furto_dataframe.RUBRICA.value_counts().head(20)

Mantendo apenas dados relativos a roubo ou furto de veículos

In [ ]:
rubricas = ["Roubo (art. 157) - VEICULO", "A.I.-Roubo (art. 157) - VEICULO", "Roubo (art. 157) - RESIDENCIA", "Furto (art. 155) - VEICULO", "Furto qualificado (art. 155, §4o.) - VEICULO", "Furto qualificado (art. 155, §4o.) - RESIDENCIA"]
roubo_furto_dataframe = roubo_furto_dataframe[roubo_furto_dataframe.RUBRICA.isin(rubricas)]
print(f'O dataset de furto e roubo de veículos possui {roubo_furto_dataframe.shape[0]} linhas e {roubo_furto_dataframe.shape[1]} colunas.')

In [ ]:
roubo_furto_dataframe.RUBRICA.value_counts()

Criando a coluna 'TIPODECRIME' para indicar se trata-se de roubo ou furto

In [ ]:
roubo_furto_dataframe['TIPODECRIME'] = 'Furto'
roubo = ['Roubo (art. 157) - VEICULO', 'A.I.-Roubo (art. 157) - VEICULO', 'Roubo (art. 157) - RESIDENCIA']
roubo_furto_dataframe.loc[roubo_furto_dataframe['RUBRICA'].isin(roubo), 'TIPODECRIME'] = 'Roubo'

In [ ]:
roubo_furto_dataframe.columns

Mantendo apenas as colunas mais significativas para a análise

In [ ]:
roubo_furto_dataframe = roubo_furto_dataframe[['DATAOCORRENCIA','PERIDOOCORRENCIA','CIDADE','DESCRICAOLOCAL','DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO','TIPODECRIME','LATITUDE','LONGITUDE']]
roubo_furto_dataframe.shape

In [ ]:
roubo_furto_dataframe.info()

In [ ]:
roubo_furto_dataframe.describe

## Analisando campos 'Latitude' e 'Longitude'

In [ ]:
lat_lon_nulas = (roubo_furto_dataframe['LATITUDE'].isna() & roubo_furto_dataframe['LONGITUDE'].isna())
print(f'{roubo_furto_dataframe[lat_lon_nulas].shape[0]} registros com latitude e longitude nulas')

In [ ]:
roubo_furto_dataframe['LONGITUDE'].describe()

In [ ]:
roubo_furto_dataframe['LATITUDE'].describe()

## Analisando campo 'período da ocorrência'

In [ ]:
roubo_furto_dataframe.PERIDOOCORRENCIA.value_counts()

## Analisando o campo 'Local'

In [ ]:
roubo_furto_dataframe.DESCRICAOLOCAL.value_counts()

Criando coluna 'LOCAL' com dados do local agrupados por locais semelhantes (maior granularidade)

In [ ]:
roubo_furto_dataframe.loc[:,'LOCAL'] = 'Outros'

residencia = ['Residência', 'Condominio Residencial', 'Garagem ou abrigo de residência', 'Garagem coletiva de prédio']
via_publica = ['Via Pública', 'Via pública', 'Favela']
comercio = ['Comércio e serviços', 'Centro Comerc./Empresarial', 'Estabelecimento industrial', 'Restaurante e afins',
            'Lazer e recreação', 'Estabelecimento de ensino', 'Shopping Center', 'Saúde', 'Serviços e bens públicos',
            'Templo e afins', 'Condominio Comercial', 'Hospedagem', 'Escritório', 'Estabelecimento bancário'
            'Entidade assistencial', 'Repartição Pública']
rodovia = ['Estrada de ferro','Rodovia/Estrada']
rural = ['Unidade rural']
veiculo_movimento = ['Veículo em movimento']
estacionamento = ['Estacionamento com vigilância', 'Estacionamento público','Estacionamento particular', 'Terminal/Estação']

roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(residencia), 'LOCAL'] = 'Residência'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(via_publica), 'LOCAL'] = 'Via Pública'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(comercio), 'LOCAL'] = 'Comércio e Serviços'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(rodovia), 'LOCAL'] = 'Estrada'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(rural), 'LOCAL'] = 'Zona Rural'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(veiculo_movimento), 'LOCAL'] = 'Veículo em movimento'
roubo_furto_dataframe.loc[roubo_furto_dataframe['DESCRICAOLOCAL'].isin(estacionamento), 'LOCAL'] = 'Estacionamento'

In [ ]:
roubo_furto_dataframe['LOCAL'].value_counts()

## Analisando o campo 'Cor'

In [ ]:
roubo_furto_dataframe.DESCR_COR_VEICULO.value_counts()

In [ ]:
print(f'{roubo_furto_dataframe.DESCR_COR_VEICULO.isna().sum()} registros com cores nulas')

Transformando cores nulas em 'Não Informado'

In [ ]:
roubo_furto_dataframe.loc[:,'DESCR_COR_VEICULO'] = roubo_furto_dataframe.DESCR_COR_VEICULO.fillna('Não Informado')
print(f'{roubo_furto_dataframe.DESCR_COR_VEICULO.value_counts()["Não Informado"]} registros com cor não informada')

## Analisando o campo 'tipo do veículo'

In [ ]:
roubo_furto_dataframe.DESCR_TIPO_VEICULO.value_counts()

In [ ]:
print(f'{roubo_furto_dataframe.DESCR_TIPO_VEICULO.isna().sum()} registros com tipo veículo nulo')

Transformando tipo veículo nulo e 'INEXIST.' em 'Não Informado'

In [ ]:
roubo_furto_dataframe.DESCR_TIPO_VEICULO = roubo_furto_dataframe.DESCR_TIPO_VEICULO.fillna('Não Informado')
roubo_furto_dataframe = roubo_furto_dataframe.replace({'DESCR_TIPO_VEICULO': {'INEXIST.': 'Não Informado'}})
roubo_furto_dataframe.DESCR_TIPO_VEICULO.value_counts()

Pela definição contida no Código de Trânsito Brasileiro:

AUTOMÓVEL - veículo automotor destinado ao transporte de passageiros, com capacidade
para até oito pessoas, exclusive o condutor

CAMIONETA – veículo misto destinado ao transporte de passageiros e carga no mesmo compartimento

CAMINHONETE – veículo destinado ao transporte de carga com peso bruto total de até três mil e quinhentos quilogramas

UTILITÁRIO – veículo misto caracterizado pela versatilidade do seu uso, inclusive fora de estrada

MOTONETA - veículo automotor de duas rodas, dirigido por condutor em posição sentada

MOTOCICLETA - veículo automotor de duas rodas, com ou sem side-car, dirigido por
condutor em posição montada.

MICROÔNIBUS - veículo automotor de transporte coletivo com capacidade para até vinte
passageiros

ÔNIBUS - veículo automotor de transporte coletivo com capacidade para mais de vinte
passageiros, ainda que, em virtude de adaptações com vista à maior comodidade destes, transporte
número menor.

CAMINHÃO-TRATOR - veículo automotor destinado a tracionar ou arrastar outro

TRATOR - veículo automotor construído para realizar trabalho agrícola, de construção e
pavimentação e tracionar outros veículos e equipamentos.

REBOQUE - veículo destinado a ser engatado atrás de um veículo automotor.

SEMI-REBOQUE - veículo de um ou mais eixos que se apóia na sua unidade tratora ou é a ela
ligado por meio de articulação.

CICLOMOTOR - veículo de duas ou três rodas, provido de um motor de combustão interna,
cuja cilindrada não exceda a cinqüenta centímetros cúbicos (3,05 polegadas cúbicas) e cuja velocidade
máxima de fabricação não exceda a cinqüenta quilômetros por hora.

## Analisando os campos 'ano do modelo' e 'ano da fabricação'

Convertendo os anos zerados para nulo

In [ ]:
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_FABRICACAO' : {0 : np.NaN}})
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_MODELO' : {0 : np.NaN}})

In [ ]:
print(f'{roubo_furto_dataframe[(roubo_furto_dataframe.ANO_FABRICACAO.isna()) | (roubo_furto_dataframe.ANO_MODELO.isna())].shape[0]} registros de ano fabricação ou ano modelo nulos')

Analisando a relação do ano do modelo com o ano de fabricação, percebe-se que são iguais ou com 1 ano de diferença.

In [ ]:
frequencia = pd.crosstab(roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 2000].ANO_MODELO,
                         roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 2000].ANO_FABRICACAO)
frequencia

In [ ]:
anos_iguais = roubo_furto_dataframe[(roubo_furto_dataframe.ANO_FABRICACAO != 0) & (roubo_furto_dataframe.ANO_MODELO == roubo_furto_dataframe.ANO_FABRICACAO)].shape[0]
print(f'{anos_iguais} veículos com o ano do modelo igual ao ano de fabricação')
anos_iguais_mais_um = roubo_furto_dataframe[(roubo_furto_dataframe.ANO_FABRICACAO != 0) & (roubo_furto_dataframe.ANO_MODELO == roubo_furto_dataframe.ANO_FABRICACAO + 1)].shape[0]
print(f'{anos_iguais_mais_um} veículos com o ano do modelo igual ao ano posterior ao ano de fabricação')

Como a maioria dos veículos possuem o ano modelo posterior ao ano de frabricação, vamos preencher os anos do modelo nulos com o ano posterior ao ano da fabricação
Preenchendo os anos de fabricação nulos com o ano anterior ao ano do modelo

In [ ]:
roubo_furto_dataframe['ANO_MODELO'] =  roubo_furto_dataframe['ANO_MODELO'].fillna(roubo_furto_dataframe['ANO_FABRICACAO'] + 1)

In [ ]:
print(f'{roubo_furto_dataframe[(roubo_furto_dataframe.ANO_FABRICACAO.isna()) | (roubo_furto_dataframe.ANO_MODELO.isna())].shape[0]} registros de ano fabricação ou ano modelo nulos')

In [ ]:
roubo_furto_dataframe['ANO_FABRICACAO'] =  roubo_furto_dataframe['ANO_FABRICACAO'].fillna(roubo_furto_dataframe['ANO_MODELO'] - 1)

In [ ]:
print(f'{roubo_furto_dataframe[(roubo_furto_dataframe.ANO_FABRICACAO.isna()) | (roubo_furto_dataframe.ANO_MODELO.isna())].shape[0]} registros de ano fabricação ou ano modelo nulos')

Convertendo as colunas de ano modelo e ano fabricação para o tipo inteiro

In [ ]:
roubo_furto_dataframe.ANO_FABRICACAO = roubo_furto_dataframe.ANO_FABRICACAO.fillna(0)
roubo_furto_dataframe.ANO_MODELO = roubo_furto_dataframe.ANO_MODELO.fillna(0)
roubo_furto_dataframe.ANO_FABRICACAO = roubo_furto_dataframe.ANO_FABRICACAO.astype(int)
roubo_furto_dataframe.ANO_MODELO = roubo_furto_dataframe.ANO_MODELO.astype(int)

Verificando e removendo outliers

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(x = 'ANO_FABRICACAO', data = roubo_furto_dataframe[roubo_furto_dataframe.ANO_FABRICACAO > 0], orient = 'h')
plt.xlabel('Ano fabricação', fontsize=14)
plt.show()

Transformando outliers em zero (nulos)

In [ ]:
roubo_furto_dataframe[roubo_furto_dataframe.ANO_FABRICACAO < 1950].ANO_FABRICACAO.value_counts()

Analisando os outliers, percebe-se que os valores estão incorretos. Eles são transformados em zeros (nulos).

In [ ]:
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_FABRICACAO' : {201 : 0}})
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_FABRICACAO' : {1918 : 0}})

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(x = ['ANO_FABRICACAO'], data = roubo_furto_dataframe[roubo_furto_dataframe.ANO_FABRICACAO > 0], orient = 'h')
plt.xlabel('Ano fabricação', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(x = 'ANO_MODELO', data = roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 0], orient = 'h')
plt.xlabel('Ano modelo', fontsize=14)
plt.show()

In [ ]:
roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO < 1950].ANO_MODELO.value_counts()

In [ ]:
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_MODELO' : {202 : 0}})
roubo_furto_dataframe = roubo_furto_dataframe.replace({'ANO_MODELO' : {1919 : 0}})

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(x = 'ANO_MODELO', data = roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 0], orient = 'h')
plt.xlabel('Ano fabricação', fontsize=14)
plt.show()

In [ ]:
freq_ano_modelo = roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 0].ANO_MODELO.value_counts()
plt.figure(figsize=(20,8))
sns.barplot(x=freq_ano_modelo.index, y=freq_ano_modelo)
plt.xticks(rotation= 90)
plt.xlabel('Ano Modelo')
plt.ylabel('Frequência')
plt.title('Distribuição de Frequências - Ano Modelo do Veículo - Roubo/furto de veículos 2019 SP')
plt.show()

In [ ]:
roubo_furto_dataframe[roubo_furto_dataframe.ANO_MODELO > 0].ANO_MODELO.describe()

Criando a coluna 'Faixa ano modelo' com os intervalos de anos correspondentes aos quartis.

In [ ]:
classes = [1951, 2008, 2013, 2016, 2020]
labels = ['1951-2008', '2008-2013', '1013-2016', '2016-2020']
roubo_furto_dataframe['FAIXA_ANO_MODELO'] = pd.cut(x = roubo_furto_dataframe.ANO_MODELO,
        bins = classes,
        labels = labels,
        include_lowest = True).values.add_categories('Não Informado')
print(f'A coluna faixa ano modelo possui {roubo_furto_dataframe[roubo_furto_dataframe.FAIXA_ANO_MODELO.isna()].shape[0]} dados nulos')
roubo_furto_dataframe.FAIXA_ANO_MODELO = roubo_furto_dataframe.FAIXA_ANO_MODELO.fillna('Não Informado')

## Analisando o campo 'Data da ocorrência'

Mantendo apenas as ocorrências com data posterior à 01/08/2019

In [ ]:
roubo_furto_dataframe['DATAOCORRENCIA2'] = roubo_furto_dataframe['DATAOCORRENCIA']
roubo_furto_dataframe.DATAOCORRENCIA2 = pd.to_datetime(roubo_furto_dataframe.DATAOCORRENCIA2, format="%Y-%m-%d", errors='coerce')
ocorrencias_antigas = roubo_furto_dataframe[roubo_furto_dataframe.DATAOCORRENCIA2 < pd.Timestamp(2019,8,1)]
ocorrencias_erradas = roubo_furto_dataframe[roubo_furto_dataframe.DATAOCORRENCIA2 == pd.NaT]
print(f'Existem {ocorrencias_antigas.shape[0]} ocorrências com data anterior à 01/08/2019')
print(f'Existem {ocorrencias_erradas.shape[0]} ocorrências com data inválida')
roubo_furto_dataframe = roubo_furto_dataframe[roubo_furto_dataframe.DATAOCORRENCIA2 >= pd.Timestamp(2019,8,1)]
roubo_furto_dataframe = roubo_furto_dataframe.drop(['DATAOCORRENCIA2'], axis = 'columns')

In [ ]:
frequencia = roubo_furto_dataframe.DATAOCORRENCIA.value_counts()
ocorrencias = pd.DataFrame({'Frequência' : frequencia})
ocorrencias.sort_values(ascending = False, by = 'Frequência')

Criando coluna indicando o dia da semana da ocorrência (Monday=0, Sunday=6)

In [ ]:
dias_da_semana = {
    0: "Segunda-feira",
    1: "Terça-feira",
    2: "Quarta-feira",
    3: "Quinta-feira",
    4: "Sexta-feira",
    5: "Sábado",
    6: "Domingo"
}
roubo_furto_dataframe['DATAOCORRENCIA'] = pd.to_datetime(roubo_furto_dataframe.DATAOCORRENCIA, format="%Y-%m-%d", errors='coerce')
roubo_furto_dataframe['DIADASEMANA'] = roubo_furto_dataframe.DATAOCORRENCIA.dt.dayofweek
roubo_furto_dataframe['DIADASEMANADESC'] = roubo_furto_dataframe['DIADASEMANA'].map(dias_da_semana)
roubo_furto_dataframe.DIADASEMANADESC.value_counts()

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.figure(figsize=(20,8))
sns.lineplot(x=ocorrencias.index,y=ocorrencias['Frequência'])
plt.title('Roubo/Furto de veículos de Agosto à Dezembro de 2019', fontSize = 16)
plt.xlabel('Data da ocorrência')
plt.ylabel('Quantidade de veículos')
plt.show()

Adicionando coluna indicando se a data é feriado, é véspera de feriado, ou é pós feriado

In [ ]:
feriados = pd.read_csv('../input/feriados-e-dias-da-semana-brasil/feriados.csv', delimiter=',', parse_dates=['Data'])
feriados.drop(columns=['Dia da Semana', 'Feriado'], inplace=True)
feriados.drop_duplicates(inplace=True)
feriados['IND_FERIADO'] = 1

In [ ]:
roubo_furto_dataframe = pd.merge(roubo_furto_dataframe, feriados, how='left', left_on='DATAOCORRENCIA', right_on='Data').drop(columns=['Data'])
roubo_furto_dataframe['IND_FERIADO'] = roubo_furto_dataframe['IND_FERIADO'].fillna(0).astype('uint8')
roubo_furto_dataframe['IND_FERIADO'].value_counts()
roubo_furto_dataframe[roubo_furto_dataframe['IND_FERIADO'] == 1]['DATAOCORRENCIA'].value_counts()

In [ ]:
pre_feriados = (feriados['Data'] + np.timedelta64(-1, 'D')).to_frame()
pre_feriados['IND_PRE_FERIADO'] = 1

In [ ]:
roubo_furto_dataframe = pd.merge(roubo_furto_dataframe, pre_feriados, how='left', left_on='DATAOCORRENCIA', right_on='Data').drop(columns=['Data'])
roubo_furto_dataframe['IND_PRE_FERIADO'] = roubo_furto_dataframe['IND_PRE_FERIADO'].fillna(0).astype('uint8')
roubo_furto_dataframe[roubo_furto_dataframe['IND_PRE_FERIADO'] == 1]['DATAOCORRENCIA'].value_counts()

In [ ]:
pos_feriados = (feriados['Data'] + np.timedelta64(1, 'D')).to_frame()
pos_feriados['IND_POS_FERIADO'] = 1

In [ ]:
roubo_furto_dataframe = pd.merge(roubo_furto_dataframe, pos_feriados, how='left', left_on='DATAOCORRENCIA', right_on='Data').drop(columns=['Data'])
roubo_furto_dataframe['IND_POS_FERIADO'] = roubo_furto_dataframe['IND_POS_FERIADO'].fillna(0).astype('uint8')
roubo_furto_dataframe[roubo_furto_dataframe['IND_POS_FERIADO'] == 1]['DATAOCORRENCIA'].value_counts()

## Analisando o campo 'Marca/Modelo'

In [ ]:
roubo_furto_dataframe.DESCR_MARCA_VEICULO.value_counts().head(20)

In [ ]:
print(f'{roubo_furto_dataframe[roubo_furto_dataframe.DESCR_MARCA_VEICULO.isna()].shape[0]} registros de marca modelo nulos')

## Analisando o campo 'Cidades'

In [ ]:
print(f'{len(roubo_furto_dataframe.CIDADE.unique())} cidades do Estado de São Paulo')
print(f'{roubo_furto_dataframe[roubo_furto_dataframe.CIDADE.isna()].shape[0]} registros de cidade nulos')

In [ ]:
roubo_furto_dataframe.CIDADE.value_counts(normalize = True).head(10)

Removendo os registros de cidade nulos

In [ ]:
print(f'Removendo {roubo_furto_dataframe.CIDADE.isna().sum()} linhas com dados nulos')
roubo_furto_dataframe.CIDADE.dropna(inplace = True)

## Contabilizando campos nulos ou com valores vagos

* Marca modelo: campos nulos
* Ano modelo e ano fabricação: campos zerados
* Tipo veículo: campos 'Não Informado'
* Cor: campos 'Não Informado'
* Local: campos 'Outros'
* Período ocorrência: campos 'EM HORA INCERTA'

In [ ]:
tem_algum_dado_nulo = (roubo_furto_dataframe['DESCR_COR_VEICULO'] == 'Não Informado') | (roubo_furto_dataframe['DESCR_MARCA_VEICULO'].isna()) | (roubo_furto_dataframe['ANO_MODELO'] == 0) | (roubo_furto_dataframe['ANO_FABRICACAO'] == 0) | (roubo_furto_dataframe['DESCR_TIPO_VEICULO'] == 'Não Informado') | (roubo_furto_dataframe['DESCRICAOLOCAL'] == 'Outros') | (roubo_furto_dataframe['PERIDOOCORRENCIA'] == 'EM HORA INCERTA')
print(f'\nTotal de linhas com algum dado nulo: {roubo_furto_dataframe[tem_algum_dado_nulo].shape[0]} de {roubo_furto_dataframe.shape[0]}')
print(f'\nTotal de linhas sem nenhum dado nulo: {roubo_furto_dataframe.shape[0] - roubo_furto_dataframe[tem_algum_dado_nulo].shape[0]}')

# Analisando roubo/furto de veículos em SP 2019 junto com dados do IBGE

Carregando a frota de SP por Município

In [ ]:
frota_ano_sp = pd.read_csv('../input/frota-de-veculos-2019-sp/I_Frota_por_UF_Municipio_Marca_e_Modelo_Ano_Dezembro_2019.csv', dtype=object, sep=';', encoding='utf8')

In [ ]:
frota_ano_sp = frota_ano_sp[['Município','Qtd. Veículos']]
frota_ano_sp.rename(columns={'Município':'municipio',
                          'Qtd. Veículos':'qtde_veiculos'}, inplace=True)
frota_ano_sp = frota_ano_sp.groupby(['municipio'], as_index=False).count()

In [ ]:
print(f'A frota de SP possui {frota_ano_sp.qtde_veiculos.sum()} veículos (dados do Denatran)')

In [ ]:
frota_ano_sp.sort_values(by='qtde_veiculos',ascending=False).head()

Agregando a quantidade de roubos/furtos por município de SP

In [ ]:
roubo_furto_dataframe['qtde_roubos'] = 1
df_roubos_agregado = roubo_furto_dataframe.groupby(['CIDADE'], as_index=False).count()
df_roubos_agregado.rename(columns={'CIDADE':'municipio'}, inplace=True)
df_roubos_agregado = df_roubos_agregado[['municipio','qtde_roubos']]
df_roubos_agregado.sort_values(by='qtde_roubos',ascending=False).head()

Compatibilizando o nome dos municípios entre os dataframes de frota e de roubo/furto de veículos

In [ ]:
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.PAULO', 'SAO PAULO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANDRE', 'SANTO ANDRE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.BERNARDO DO CAMPO', 'SAO BERNARDO DO CAMPO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOSE DOS CAMPOS', 'SAO JOSE DOS CAMPOS')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.VICENTE', 'SAO VICENTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.BARBARA D OESTE', 'SANTA BARBARA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CARLOS', 'SAO CARLOS')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ISABEL', 'SANTA ISABEL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.LOURENCO DA SERRA', 'SAO LOURENCO DA SERRA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ROQUE', 'SAO ROQUE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CAETANO DO SUL', 'SAO CAETANO DO SUL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOSE DO RIO PRETO', 'SAO JOSE DO RIO PRETO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANTONIO DE POSSE', 'SANTO ANTONIO DE POSSE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANTONIO DO JARDIM', 'SANTO ANTONIO DO JARDIM')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.SEBASTIAO', 'SAO SEBASTIAO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOSE DO RIO PARDO', 'SAO JOSE DO RIO PARDO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.SEBASTIAO', 'SAO SEBASTIAO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.MARIA DA SERRA', 'SANTA MARIA DA SERRA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CRUZ DAS PALMEIRAS', 'SANTA CRUZ DAS PALMEIRAS')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.SEBASTIAO', 'SAO SEBASTIAO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.GERTRUDES', 'SANTA GERTRUDES')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANTONIO DA ALEGRIA', 'SANTO ANTONIO DA ALEGRIA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANTONIO DE ARACANGUA', 'SANTO ANTONIO DO ARACANGUA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ANTONIO DO PINHAL', 'SANTO ANTONIO DO PINHAL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.BENTO DO SAPUCAI', 'SAO BENTO DO SAPUCAI')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.BRANCA', 'SANTA BRANCA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CLARA D OESTE', 'SANTA CLARA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CRUZ DA ESPERANCA', 'SANTA CRUZ DA ESPERANCA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.CRUZ DO RIO PARDO', 'SANTA CRUZ DO RIO PARDO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ERNESTINA', 'SANTA ERNESTINA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.FE DO SUL', 'SANTA FE DO SUL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.FRANCISCO', 'SAO FRANCISCO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOAO DA BOA VISTA', 'SAO JOAO DA BOA VISTA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOAQUIM DA BARRA', 'SAO JOAQUIM DA BARRA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.JOSE DA BELA VISTA', 'SAO JOSE DA BELA VISTA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.LUIS DO PARAITINGA', 'SAO LUIZ DO PARAITINGA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.MANUEL', 'SAO MANUEL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.MIGUEL ARCANJO', 'SAO MIGUEL ARCANJO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.PEDRO', 'SAO PEDRO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.PEDRO DO TURVO', 'SAO PEDRO DO TURVO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.RITA D OESTE', 'SANTA RITA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.RITA PASSA QUATRO', 'SANTA RITA DO PASSA QUATRO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.ROSA DE VITERBO', 'SANTA ROSA DE VITERBO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('S.SIMAO', 'SAO SIMAO')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('APARECIDA D OESTE', 'APARECIDA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('ESPIRITO STO. PINHAL', 'ESPIRITO SANTO DO PINHAL')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('ESTRELA D OESTE', 'ESTRELA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('PALMEIRA D OESTE', 'PALMEIRA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('MIRANTE PARANAPANEMA', 'MIRANTE DO PARANAPANEMA')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('MOGI GUACU', 'MOGI-GUACU')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('MOGI MIRIM', 'MOGI-MIRIM')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('PIRAPORA BOM JESUS', 'PIRAPORA DO BOM JESUS')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('SANTA BARBARA D OESTE', 'SANTA BARBARA D\'OESTE')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('SANTO ANTONIO DO JARDIM', 'SANTO ANTONIO DO JARDIM')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('EMBU DAS ARTES', 'EMBU')

Gerando a taxa de veículos roubados/furtados a cada 1000 veículos por município

In [ ]:
df_roubos_agregado = pd.merge(df_roubos_agregado, frota_ano_sp,  how='left', left_on='municipio', right_on = 'municipio')
df_roubos_agregado['tx_roubo_frota'] = (df_roubos_agregado['qtde_roubos']/df_roubos_agregado['qtde_veiculos'])*1000

In [ ]:
df_roubos_agregado.sort_values(by='tx_roubo_frota', ascending=False)

Carregando código do IBGE de cada município

In [ ]:
df_municipio = pd.read_csv('../input/municipios-sp-ibge/municipios.csv', dtype=object, sep=';', encoding='iso-8859-1')

In [ ]:
df_municipio = pd.DataFrame (df_municipio, columns = ['cod_municipio_ibge','sig_uf','municipio'])
df_municipio_sp = df_municipio[df_municipio['sig_uf'] == 'SP']

Compatibilizando o nome dos municípios

In [ ]:
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('EMBU', 'EMBU DAS ARTES')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('MOGI-GUACU', 'MOGI GUACU')
df_roubos_agregado['municipio'] = df_roubos_agregado['municipio'].str.replace('MOGI-MIRIM', 'MOGI MIRIM')

In [ ]:
del df_municipio_sp['sig_uf']

In [ ]:
df_roubos_agregado = pd.merge(df_roubos_agregado, df_municipio_sp,  how='left', left_on='municipio', right_on = 'municipio')

In [ ]:
df_roubos_agregado.head()

Lendo dados dos IBGE

In [ ]:
df_municipio_ibge = pd.read_csv('../input/municipios-sp-ibge/municipios_ibge.csv', dtype=object, sep=';', encoding='iso-8859-1')

In [ ]:
df_municipio_ibge['receitas_realizadas'] = df_municipio_ibge['receitas_realizadas'].astype('float').round()
df_municipio_ibge['despesas_empenhadas'] = df_municipio_ibge['despesas_empenhadas'].astype('float').round()

In [ ]:
df_roubos_agregado = pd.merge(df_roubos_agregado, df_municipio_ibge,  how='left', left_on='cod_municipio_ibge', right_on = 'cod_municipio_ibge')

In [ ]:
df_roubos_agregado.info()

In [ ]:
df_roubos_agregado = df_roubos_agregado[df_roubos_agregado['mortalizade_infantil_por_mil']!= '-']
df_roubos_agregado['mortalizade_infantil_por_mil'] = df_roubos_agregado['mortalizade_infantil_por_mil'].astype('float')

df_roubos_agregado['população'] = pd.to_numeric(df_roubos_agregado['população'], errors='coerce')
df_roubos_agregado.dropna(inplace=True)
df_roubos_agregado['população'] = df_roubos_agregado['população'].astype('int64')

df_roubos_agregado['idhm'] = df_roubos_agregado['idhm'].astype('float')
df_roubos_agregado['densidade_demografica'] = df_roubos_agregado['densidade_demografica'].astype('float')
df_roubos_agregado['escolarizacao_06_a_14_%'] = df_roubos_agregado['escolarizacao_06_a_14_%'].astype('float')
df_roubos_agregado['pib_percapita'] = df_roubos_agregado['pib_percapita'].astype('float')

Correlação entre a taxa de roubo/furto de veículos e IDH, Receita, Mortalidade Infatil, Densidade demográfica, Eslocarização e PIB de cada município de SP

In [ ]:
df_roubos_agregado_corr = df_roubos_agregado[['tx_roubo_frota', 'densidade_demografica', 'escolarizacao_06_a_14_%', 'idhm', 'mortalizade_infantil_por_mil', 'receitas_realizadas', 'despesas_empenhadas', 'pib_percapita']]
matriz_corr = df_roubos_agregado_corr.corr()
plt.figure(figsize=(10,5))
sns.heatmap(matriz_corr, annot=True)
plt.show()

In [ ]:
mask = np.triu(np.ones_like(matriz_corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(matriz_corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

# Analisando padrões nas ocorrências de roubo/furto através de agrupamento

In [ ]:
scaler = StandardScaler()

In [ ]:
def kmeans(numero_de_clusters, dados):
  modelo = KMeans(n_clusters=numero_de_clusters)
  modelo.fit(dados)
  return [numero_de_clusters, modelo.inertia_]

In [ ]:
#plotando os dados do dataframe no mapa de São Paulo e com a cor do grupo associado
def plotando_no_mapa_todos_grupos(dados):
    #Importanto o mapa de municípios do Estado de SP
    sp = gpd.read_file('../input/sp-municpios-geopandas/35MUE250GC_SIR.shp')

    geometry = [Point(x) for x in zip(dados['LONGITUDE'], dados['LATITUDE'])]
    crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}
    geo_dados = gpd.GeoDataFrame(dados, crs = crs, geometry = geometry)
    base = sp.plot(color='white', edgecolor='black', figsize=(20, 12))
    geo_dados.plot(ax=base, figsize=(20, 12), alpha=0.8, column='GRUPO_DESC',categorical=True,
               legend=True, legend_kwds={'fontsize':14}, markersize=2)
    base.axis('off')
    base.set_title('Grupos de ocorrências de roubo e furto em SP',fontsize=20)
    plt.tight_layout()
    plt.savefig('analise_roubos_sp.png',dpi=300)

In [ ]:
#plotando os dados de apenas um grupo do dataframe no mapa de São Paulo
def plotando_grupo_no_mapa(dataframe, grupo, grupos):
    #Importanto o mapa de municípios do Estado de SP
    sp = gpd.read_file('../input/sp-municpios-geopandas/35MUE250GC_SIR.shp')

    #filtrando o grupo escolhido
    filtro = dataframe['grupo'] == grupo
    dados = dataframe[filtro]

    fig = plt.figure(figsize=(50, 30))

    ax1 = plt.subplot2grid((20, 40), (0, 0), colspan=36, rowspan=20)
    ax2 = plt.subplot2grid((20, 40), (0, 36), colspan=4, rowspan=10)

    geometry = [Point(x) for x in zip(dados['LONGITUDE'], dados['LATITUDE'])]
    crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}
    geo_dados = gpd.GeoDataFrame(dados, crs = crs, geometry = geometry)
    base = sp.plot(color='white', edgecolor='black', ax=ax1)
    geo_dados.plot(ax=base, figsize=(20, 12), alpha=0.8, column='grupo',categorical=True,
               legend=False, legend_kwds={'fontsize':14}, markersize=5)
    base.axis('off')
    base.set_title(f'Grupos de ocorrências de roubo e furto em SP - Grupo {grupo}',fontsize=24)
    grupos.iloc[grupo][grupos.iloc[grupo] > 0].sort_values().plot.barh(ax=ax2, fontsize=20)
    fig.savefig(f'roubo_sp_grupo{grupo}.png')

In [ ]:
#plotando os dados do dataframe no mapa de São Paulo e com a cor do grupo associado e guardando em uma imagem png
def gerando_imagem_do_mapa(grupo, dataframe):
    #Importanto o mapa de municípios do Estado de SP
    sp = gpd.read_file('../input/sp-municpios-geopandas/35MUE250GC_SIR.shp')

    #filtrando o grupo escolhido
    filtro = dataframe['grupo'] == grupo
    dados = dataframe[filtro]

    geometry = [Point(x) for x in zip(dados['LONGITUDE'], dados['LATITUDE'])]
    crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}
    geo_dados = gpd.GeoDataFrame(dados, crs = crs, geometry = geometry)
    base = sp.plot(color='white', edgecolor='black', figsize=(20, 12))
    geo_dados.plot(ax=base, figsize=(20, 12), alpha=0.8, column='GRUPO_DESC',categorical=True,
               legend=True, legend_kwds={'fontsize':14}, markersize=5)
    base.axis('off')
    base.set_title('Grupos de ocorrências de roubo e furto em SP',fontsize=20)
    plt.tight_layout()
    plt.savefig('analise_roubos_sp_'+ grupo + '.png',dpi=300)

In [ ]:
#treinamento Kmeans com k variando do valor inicial ao final e análise do gráfico da inércia
def treinamento_kmeans_analise_inertia(dataframe, k_inicial, k_final):
    resultado = [kmeans(numero_de_grupos, dataframe) for numero_de_grupos in range(k_inicial, k_final)]
    resultado = pd.DataFrame(resultado,
            columns=['grupos', 'inertia'])
    resultado.inertia.plot(xticks=resultado.grupos)
    plt.show()

In [ ]:
# def optimal_number_of_clusters(wcss):
#     x1, y1 = 2, wcss[0]
#     x2, y2 = 20, wcss[len(wcss)-1]

#     distances = []
#     for i in range(len(wcss)):
#         x0 = i+2
#         y0 = wcss[i]
#         numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
#         denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
#         distances.append(numerator/denominator)

#     return distances.index(max(distances)) + 2

In [ ]:
#analisando as características que mais aparecem em cada grupo
def analisando_grupos(dataframe, modelo, columns):
    grupos = pd.DataFrame(modelo.cluster_centers_,
            columns=dataframe.columns)
    grupos.transpose().plot.bar(subplots=True,
               figsize=(20, 80),
               layout=(25,columns),
               sharex=True)
    plt.show()
    return grupos

In [ ]:
def obter_grupos(dataframe, modelo):
    grupos = pd.DataFrame(modelo.cluster_centers_,columns=dataframe.columns)
    return grupos

In [ ]:
#obter string com as caracteristicas mais marcantes do grupo
def obter_label_do_grupo(grupo, grupos):
    label = ''
    for x in grupos.iloc[grupo][grupos.iloc[grupo] > 0.0].sort_values(ascending = False).index.values:
        label = label + ', ' + x
    label = label[2:]
    return label

In [ ]:
def analisar_grupos_em_2_dimensoes_tsne(dataframe, modelo, k):

    #demora para rodar
    tsne = TSNE()
    visualizacao = tsne.fit_transform(dataframe)

    sns.set(rc={'figure.figsize': (15, 15)})
    sns.scatterplot(x=visualizacao[:, 0],
                    y=visualizacao[:, 1],
                    hue=modelo.labels_,
                    palette=sns.color_palette('Set2', k))
    plt.show()

In [ ]:
from sklearn.decomposition import PCA
def analisar_grupos_em_2_dimensoes_pca(dataframe, modelo, k):
    pca = PCA(n_components=2)
    pca.fit(dataframe)
    visualizacao = pca.transform(dataframe)

    sns.set(rc={'figure.figsize': (15, 15)})
    sns.scatterplot(x=visualizacao[:, 0],
                    y=visualizacao[:, 1],
                    hue=modelo.labels_,
                    palette=sns.color_palette('Set2', k))
    plt.show()

In [ ]:
def analisar_correlacoes(dataframe):
    matriz_corr = dataframe.corr()
    plt.figure(figsize=(10,5))
    sns.heatmap(matriz_corr, annot=True)
    plt.show()

1. ## Agrupando os veículos roubados/furtados com base no Local e no Período da Ocorrência

Desconsiderando dados nulos ou vagos e considerando apenas as colunas desejadas

In [ ]:
dados_validos = (roubo_furto_dataframe['LOCAL'] != 'Outros') & (roubo_furto_dataframe['PERIDOOCORRENCIA'] != 'EM HORA INCERTA')
roubo_locais_latlon = roubo_furto_dataframe[dados_validos]
roubo_locais_latlon = roubo_locais_latlon[['LOCAL', 'PERIDOOCORRENCIA', 'LATITUDE', 'LONGITUDE']]
roubo_locais_latlon = roubo_locais_latlon.reset_index()

Gerando dummies das colunas categóricas 'LOCAL' e 'PERIDOOCORRENCIA'

In [ ]:
locais = roubo_locais_latlon.LOCAL.str.get_dummies()
periodos = roubo_locais_latlon.PERIDOOCORRENCIA.str.get_dummies()
roubos = pd.concat([locais,periodos], axis = 1)

Escalonização dos dados

In [ ]:
roubos_escalados = scaler.fit_transform(roubos)

Gerando agrupamentos com k variando de 1 a 20 grupos

In [ ]:
treinamento_kmeans_analise_inertia(roubos_escalados,1,20)

Após análise da inércia, pelo método do cotovelo, percebemos que o valor próximo do ideal seria k = 10

In [ ]:
modelo = KMeans(n_clusters=10, random_state=50)
modelo.fit(roubos_escalados)

In [ ]:
analisar_grupos_em_2_dimensoes_tsne(roubos,modelo,10)

In [ ]:
classificacao = pd.DataFrame(modelo.labels_, columns = ['grupo'])
classificacao.grupo.value_counts()

In [ ]:
analisando_grupos(roubos, modelo, 5)

In [ ]:
grupos_labels = {
    0 : "Via pública, de madrugada",
    1 : "Via pública, à noite",
    2 : "Via pública, pela manhã",
    3 : "Via pública, à tarde",
    4 : "Zona Rural, à noite",
    5 : "Veículo em movimento, à noite",
    6 : "Residência, de madrugada",
    7 : "Estacionamento, à tarde",
    8 : "Comércio e serviço, de madrugada",
    9 : "Estrada, de madrugada"
}

In [ ]:
roubo_locais_latlon_grupo = pd.concat([roubo_locais_latlon,classificacao], axis = 1)
roubo_locais_latlon_grupo['GRUPO_DESC'] = roubo_locais_latlon_grupo['grupo'].map(grupos_labels)
roubo_locais_latlon_grupo['GRUPO_DESC'].value_counts()

In [ ]:
plotando_no_mapa_todos_grupos(roubo_locais_latlon_grupo)


## Agrupando os veículos roubados/furtados com base no Local, no Período da Ocorrência, no tipo de veículo

In [ ]:
dados_validos = (roubo_furto_dataframe['LOCAL'] != 'Outros') & (roubo_furto_dataframe['PERIDOOCORRENCIA'] != 'EM HORA INCERTA') & (roubo_furto_dataframe['DESCR_TIPO_VEICULO'] != 'Não Informado')
roubo_locais_tipo_latlon = roubo_furto_dataframe[dados_validos]
roubo_locais_tipo_latlon = roubo_locais_tipo_latlon[['LOCAL', 'PERIDOOCORRENCIA','TIPODECRIME','DESCR_TIPO_VEICULO','LATITUDE', 'LONGITUDE']]
roubo_locais_tipo_latlon = roubo_locais_tipo_latlon.reset_index()

In [ ]:
locais = roubo_locais_tipo_latlon.LOCAL.str.get_dummies()
periodos = roubo_locais_tipo_latlon.PERIDOOCORRENCIA.str.get_dummies()
tipocrimes = roubo_locais_tipo_latlon.TIPODECRIME.str.get_dummies()
tipoveiculos = roubo_locais_tipo_latlon.DESCR_TIPO_VEICULO.str.get_dummies()
roubos = pd.concat([locais,periodos, tipocrimes, tipoveiculos], axis = 1)

In [ ]:
roubos_escalados = scaler.fit_transform(roubos)

In [ ]:
treinamento_kmeans_analise_inertia(roubos_escalados,8,41)

Após análise da inércia, percebemos que o valor próximo do ideal seria k = 17

In [ ]:
modelo = KMeans(n_clusters=17, random_state=50)
modelo.fit(roubos_escalados)

In [ ]:
classificacao = pd.DataFrame(modelo.labels_, columns = ['grupo'])
classificacao.grupo.value_counts()

In [ ]:
analisando_grupos(roubos, modelo, 3)

In [ ]:
grupos_labels = {
    0 : "Furto de automóvel e caminhote em via pública pela manhã",
    1 : "Roubo de ônibus em zona rural ou estrada pela manhã e madrugada",
    2 : "Roubo de automóvel em via pública, à noite",
    3 : "Furto de automóvel e caminhote em via pública à tarde",
    4 : "Furto de motociclo e automóvel em estacionamento",
    5 : "Roubo de reboque e caminhão em estrada de madrugada",
    6 : "Roubo em via pública de madrugada",
    7 : "Roubo de trator em Comércio e Serviços e na Zona Rural",
    8 : "Furto em residência de madrugada",
    9 : "Roubo de caminhonete e caminhão na Zona Rural",
    10 : "Roubo de motociclo à noite em via pública",
    11 : "Roubo de motociclo em movimento à noite",
    12 : "Roubo de utilitário em Comércio e Serviços",
    13 : "Roubo de micro-ônibus na Zona Rural pela manhã",
    14 : "Furto de caminhão e similares em Comércio e Serviços",
    15 : "Furto de motoneta",
    16 : "Roubo de caminhão em Comércio e Serviços de madrugada",
}

In [ ]:
roubo_locais_tipo_latlon_grupo = pd.concat([roubo_locais_tipo_latlon,classificacao], axis = 1)
roubo_locais_tipo_latlon_grupo['GRUPO_DESC'] = roubo_locais_tipo_latlon_grupo['grupo'].map(grupos_labels)

In [ ]:
roubo_locais_tipo_latlon_grupo['GRUPO_DESC'].value_counts

In [ ]:
plotando_no_mapa_todos_grupos(roubo_locais_tipo_latlon_grupo)

In [ ]:
grupos = obter_grupos(roubos, modelo)
plotando_grupo_no_mapa(roubo_locais_tipo_latlon_grupo, 6, grupos)

## Agrupando os veículos roubados/furtados com base no Local, no Período da Ocorrência, no tipo de veículo, e no dia da semana, feriado e faixa do ano modelo

In [ ]:
dados_validos = (roubo_furto_dataframe['LOCAL'] != 'Outros') \
    & (roubo_furto_dataframe['PERIDOOCORRENCIA'] != 'EM HORA INCERTA') \
    & (roubo_furto_dataframe['DESCR_TIPO_VEICULO'] != 'Não Informado') \
    & (roubo_furto_dataframe['FAIXA_ANO_MODELO'] != 'Não Informado')
roubo_locais_tipo_dia = roubo_furto_dataframe[dados_validos]
roubo_locais_tipo_dia = roubo_locais_tipo_dia[['LATITUDE', 'LONGITUDE','LOCAL', 'PERIDOOCORRENCIA','TIPODECRIME','DESCR_TIPO_VEICULO', 'DIADASEMANADESC','IND_FERIADO','IND_PRE_FERIADO','IND_POS_FERIADO', 'FAIXA_ANO_MODELO']]

In [ ]:
feriado = roubo_locais_tipo_dia.IND_FERIADO
preferiado = roubo_locais_tipo_dia.IND_PRE_FERIADO
posferiado = roubo_locais_tipo_dia.IND_POS_FERIADO
anos = roubo_locais_tipo_dia.FAIXA_ANO_MODELO.str.get_dummies()
dias = roubo_locais_tipo_dia.DIADASEMANADESC.str.get_dummies()
locais = roubo_locais_tipo_dia.LOCAL.str.get_dummies()
periodos = roubo_locais_tipo_dia.PERIDOOCORRENCIA.str.get_dummies()
tipocrimes = roubo_locais_tipo_dia.TIPODECRIME.str.get_dummies()
tipoveiculos = roubo_locais_tipo_dia.DESCR_TIPO_VEICULO.str.get_dummies()
roubos = pd.concat([locais,periodos, tipocrimes, tipoveiculos,dias,feriado,preferiado,posferiado,anos], axis = 1)

In [ ]:
roubos_escalados = scaler.fit_transform(roubos)

In [ ]:
treinamento_kmeans_analise_inertia(roubos_escalados,14,38)

Após análise da inércia, percebemos que o valor próximo do ideal seria k = 21

In [ ]:
modelo = KMeans(n_clusters=21, random_state=50)
modelo.fit(roubos_escalados)

In [ ]:
analisar_grupos_em_2_dimensoes_tsne(roubos,modelo,21)

In [ ]:
classificacao = pd.DataFrame(modelo.labels_, columns = ['grupo'])
ranking_grupos = pd.DataFrame({'Total de Ocorrências' : classificacao.grupo.value_counts()})
ranking_grupos

In [ ]:
roubo_locais_tipo_dia = roubo_locais_tipo_dia.reset_index()
roubo_locais_tipo_dia_grupo = pd.concat([roubo_locais_tipo_dia,classificacao], axis = 1)

In [ ]:
#for g in range(20,20):
    grupos = obter_grupos(roubos, modelo)
    plotando_grupo_no_mapa(roubo_locais_tipo_dia_grupo, 20, grupos)